In [ ]:
# CELL 1: Environment Setup and Package Installation
print("🔧 Setting up Citizen AI Platform environment...")
print("⚡ Using T4 GPU runtime")

# Install required packages
!pip install streamlit --quiet
!pip install transformers torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install accelerate --quiet
!pip install plotly --quiet
!pip install textblob --quiet
!pip install wordcloud --quiet
!pip install matplotlib seaborn --quiet
!pip install pandas numpy --quiet
!pip install pyngrok --quiet

# Download NLTK data for TextBlob
import nltk
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True)

# Verify GPU availability
import torch
print(f"🎯 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("✅ Environment setup complete!")

🔧 Setting up Citizen AI Platform environment...
⚡ Using T4 GPU runtime
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12

In [ ]:
# CELL 2: Create Main Application File
app_code = '''
# Citizen AI - Intelligent Citizen Engagement Platform (Streamlit Version)
# Complete implementation with Granite 3.3 2B model and advanced features

import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import re
from collections import defaultdict, Counter
import plotly.graph_objects as go
import plotly.express as px
from textblob import TextBlob
import time

# Install required packages (run this in Google Colab first)
"""
!pip install streamlit transformers torch plotly textblob pandas numpy
!pip install accelerate bitsandbytes optimum

# For even faster inference, also install:
!pip install flash-attn --no-build-isolation  # Optional for CUDA

# To run: streamlit run citizen_ai_streamlit.py
"""

# Page configuration
st.set_page_config(
    page_title="Citizen AI - Intelligent Engagement Platform",
    page_icon="🏛️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        padding: 1rem;
        border-radius: 10px;
        color: white;
        text-align: center;
        margin-bottom: 2rem;
    }
    .metric-card {
        background: #f8f9fa;
        padding: 1rem;
        border-radius: 10px;
        border-left: 4px solid #667eea;
        margin: 0.5rem 0;
    }
    .chat-message {
        padding: 1rem;
        border-radius: 10px;
        margin: 0.5rem 0;
    }
    .user-message {
        background: #e3f2fd;
        border-left: 4px solid #2196f3;
    }
    .assistant-message {
        background: #f3e5f5;
        border-left: 4px solid #9c27b0;
    }
    .sidebar-info {
        background: #fff3e0;
        padding: 1rem;
        border-radius: 10px;
        border: 1px solid #ff9800;
    }
</style>
""", unsafe_allow_html=True)

class CitizenAI:
    def __init__(self):
      self.model_name = "ibm-granite/granite-3.3-2b-instruct"
      #self.model_name = "mistralai/Mistral-7B-Instruct-v0.1"


    @st.cache_resource
    def load_model(_self):
        """Load the Granite 3.3 2B model with caching"""
        try:
            with st.spinner("🔄 Loading Granite 3.3 2B model... This may take a few minutes."):
                progress_bar = st.progress(0)

                # Load tokenizer
                progress_bar.progress(25)
                tokenizer = AutoTokenizer.from_pretrained(
                    _self.model_name,
                    use_fast=True
                )
                #tokenizer = AutoTokenizer.from_pretrained(_self.model_name)

                # Check device
                device = "cuda" if torch.cuda.is_available() else "cpu"

                # Load model
                progress_bar.progress(50)
                model = AutoModelForCausalLM.from_pretrained(
                    _self.model_name,
                    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
                    device_map="auto" if device == "cuda" else None,
                    low_cpu_mem_usage=True,
                    use_cache=True
                )
                # progress_bar.progress(50)
                # model = AutoModelForCausalLM.from_pretrained(
                #   _self.model_name,
                #   torch_dtype=torch.float16,
                #   device_map="auto"
                # )

                # Move to device
                progress_bar.progress(75)
                if device == "cuda":
                    model = model.to(device)

                # Set pad token
                if tokenizer.pad_token is None:
                    tokenizer.pad_token = tokenizer.eos_token

                # Compile model for speed
                try:
                    model = torch.compile(model, mode="reduce-overhead")
                except:
                    pass

                progress_bar.progress(100)
                st.success("✅ Model loaded successfully!")

                return tokenizer, model, device

        except Exception as e:
            st.error(f"❌ Error loading model: {str(e)}")
            return None, None, None

    def analyze_sentiment(self, text):
        """Analyze sentiment of citizen input"""
        try:
            blob = TextBlob(text)
            sentiment_score = blob.sentiment.polarity

            if sentiment_score > 0.1:
                sentiment_label = "Positive"
                emoji = "😊"
            elif sentiment_score < -0.1:
                sentiment_label = "Negative"
                emoji = "😟"
            else:
                sentiment_label = "Neutral"
                emoji = "😐"

            return {
                'score': sentiment_score,
                'label': sentiment_label,
                'emoji': emoji,
                'confidence': abs(sentiment_score)
            }
        except:
            return {
                'score': 0.0,
                'label': "Neutral",
                'emoji': "😐",
                'confidence': 0.0
            }

    def extract_topics(self, text):
        """Extract key topics from citizen input"""
        topic_keywords = {
            'infrastructure': ['road', 'bridge', 'water', 'electricity', 'internet', 'transport'],
            'healthcare': ['hospital', 'doctor', 'medicine', 'health', 'clinic', 'emergency'],
            'education': ['school', 'college', 'teacher', 'student', 'education', 'learning'],
            'environment': ['pollution', 'waste', 'climate', 'green', 'clean', 'environment'],
            'safety': ['police', 'crime', 'safety', 'security', 'violence', 'emergency'],
            'economy': ['job', 'employment', 'business', 'economy', 'money', 'tax'],
            'governance': ['government', 'policy', 'law', 'regulation', 'administration', 'service']
        }

        text_lower = text.lower()
        detected_topics = []

        for topic, keywords in topic_keywords.items():
            if any(keyword in text_lower for keyword in keywords):
                detected_topics.append(topic)

        return detected_topics if detected_topics else ['general']

# Initialize session state
def initialize_session_state():
    if 'conversation_history' not in st.session_state:
        st.session_state.conversation_history = []
    if 'sentiment_data' not in st.session_state:
        st.session_state.sentiment_data = []
    if 'topics_discussed' not in st.session_state:
        st.session_state.topics_discussed = defaultdict(int)
    if 'model_loaded' not in st.session_state:
        st.session_state.model_loaded = False
    if 'tokenizer' not in st.session_state:
        st.session_state.tokenizer = None
    if 'model' not in st.session_state:
        st.session_state.model = None
    if 'device' not in st.session_state:
        st.session_state.device = None

def generate_response(user_input, user_name="Citizen"):
    """Generate response using the loaded model"""
    if not st.session_state.model_loaded:
        return "Please load the model first using the sidebar."

    try:
        ai = CitizenAI()

        # Analyze input
        sentiment = ai.analyze_sentiment(user_input)
        topics = ai.extract_topics(user_input)

        # Update topics counter
        for topic in topics:
            st.session_state.topics_discussed[topic] += 1

        # Create prompt
        system_prompt = f"""You are a helpful AI assistant for citizen engagement. Help citizens with their concerns about government services and community issues.

Citizen: {user_name}
Topics: {', '.join(topics)}
Sentiment: {sentiment['label']}

Provide a helpful, concise response (max 2-3 sentences)."""

        # Get context from recent history
        context_msg = ""
        if st.session_state.conversation_history:
            last_exchange = st.session_state.conversation_history[-1]
            context_msg = f"Previous: {last_exchange['user'][:50]}..."

        full_prompt = f"""{system_prompt}\n\nContext: {context_msg}\n\nCitizen: {user_input}\n\nAssistant:"""

        # Tokenize
        inputs = st.session_state.tokenizer(
            full_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=False
        )

        # Move to device
        inputs = {k: v.to(st.session_state.device) for k, v in inputs.items()}

        # Generate
        with torch.no_grad():
            outputs = st.session_state.model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.3,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=st.session_state.tokenizer.eos_token_id,
                eos_token_id=st.session_state.tokenizer.eos_token_id,
                use_cache=True
            )

        # Decode
        response = st.session_state.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        ).strip()

        # Fallback if empty
        if not response:
            response = f"Thank you for your concern about {topics[0] if topics else 'this matter'}. I understand your {sentiment['label'].lower()} feelings. How can I help you further?"

        # Store conversation
        conversation_entry = {
            'timestamp': datetime.now(),
            'user': user_input,
            'assistant': response,
            'sentiment': sentiment,
            'topics': topics,
            'user_name': user_name
        }

        st.session_state.conversation_history.append(conversation_entry)

        # Store sentiment data
        st.session_state.sentiment_data.append({
            'timestamp': datetime.now(),
            'sentiment_score': sentiment['score'],
            'sentiment_label': sentiment['label'],
            'topics': topics
        })

        # Limit history
        if len(st.session_state.conversation_history) > 20:
            st.session_state.conversation_history = st.session_state.conversation_history[-20:]

        if len(st.session_state.sentiment_data) > 50:
            st.session_state.sentiment_data = st.session_state.sentiment_data[-50:]

        return response

    except Exception as e:
        st.error(f"Error generating response: {str(e)}")
        return f"I understand your concern. Could you provide more specific details about what you need assistance with?"

def create_sentiment_chart():
    """Create sentiment analysis chart"""
    if not st.session_state.sentiment_data:
        fig = go.Figure()
        fig.add_annotation(text="No sentiment data available yet", x=0.5, y=0.5, showarrow=False)
        fig.update_layout(title="Sentiment Analysis Over Time", height=400)
        return fig

    df = pd.DataFrame(st.session_state.sentiment_data)
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['timestamp'],
        y=df['sentiment_score'],
        mode='lines+markers',
        name='Sentiment Score',
        line=dict(color='#667eea', width=3),
        marker=dict(size=8, color='#764ba2')
    ))

    fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Neutral")
    fig.update_layout(
        title="📈 Citizen Sentiment Over Time",
        xaxis_title="Time",
        yaxis_title="Sentiment Score",
        yaxis=dict(range=[-1, 1]),
        height=400,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    return fig

def create_topics_chart():
    """Create topics distribution chart"""
    if not st.session_state.topics_discussed:
        fig = go.Figure()
        fig.add_annotation(text="No topics discussed yet", x=0.5, y=0.5, showarrow=False)
        fig.update_layout(title="Topics Distribution", height=400)
        return fig

    topics = list(st.session_state.topics_discussed.keys())
    counts = list(st.session_state.topics_discussed.values())

    fig = go.Figure(data=[go.Bar(
        x=topics,
        y=counts,
        marker_color='#667eea',
        text=counts,
        textposition='auto',
    )])

    fig.update_layout(
        title="🏷️ Most Discussed Topics",
        xaxis_title="Topics",
        yaxis_title="Frequency",
        height=400,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    return fig

def main():
    # Initialize session state
    initialize_session_state()

    # Header
    st.markdown("""
    <div class="main-header">
        <h1>🏛️ Citizen AI - Intelligent Citizen Engagement Platform</h1>
        <p>Empowering citizens through AI-powered conversations and insights</p>
    </div>
    """, unsafe_allow_html=True)

    # Sidebar
    with st.sidebar:
        st.markdown("## 🔧 Model Management")

        if st.button("🚀 Initialize Model", type="primary", use_container_width=True):
            ai = CitizenAI()
            tokenizer, model, device = ai.load_model()

            if tokenizer and model:
                st.session_state.tokenizer = tokenizer
                st.session_state.model = model
                st.session_state.device = device
                st.session_state.model_loaded = True
                st.rerun()

        # Model status
        if st.session_state.model_loaded:
            st.success("✅ Model Ready")
            device_info = st.session_state.device if st.session_state.device else "CPU"
            st.info(f"🖥️ Device: {device_info}")
        else:
            st.warning("⚠️ Model not loaded")

        st.markdown("---")

        # Quick stats
        st.markdown("## 📊 Quick Stats")
        total_conversations = len(st.session_state.conversation_history)

        if st.session_state.sentiment_data:
            avg_sentiment = np.mean([s['sentiment_score'] for s in st.session_state.sentiment_data])
            if avg_sentiment > 0.1:
                sentiment_emoji = "😊 Positive"
            elif avg_sentiment < -0.1:
                sentiment_emoji = "😟 Negative"
            else:
                sentiment_emoji = "😐 Neutral"
        else:
            avg_sentiment = 0.0
            sentiment_emoji = "😐 Neutral"

        st.metric("Total Conversations", total_conversations)
        st.metric("Average Sentiment", f"{avg_sentiment:.2f}")
        st.markdown(f"**Mood**: {sentiment_emoji}")

        st.markdown("---")

        if st.button("🗑️ Clear All Data", type="secondary", use_container_width=True):
            st.session_state.conversation_history = []
            st.session_state.sentiment_data = []
            st.session_state.topics_discussed = defaultdict(int)
            st.success("Data cleared!")
            st.rerun()

        # Info section
        st.markdown("""
        <div class="sidebar-info">
        <strong>💡 Tips:</strong><br>
        • Initialize the model first<br>
        • Ask about government services<br>
        • Share your concerns<br>
        • Check the dashboard for insights
        </div>
        """, unsafe_allow_html=True)

    # Main content tabs
    tab1, tab2, tab3 = st.tabs(["💬 Chat", "📊 Dashboard", "📈 Analytics"])

    with tab1:
        st.markdown("## 💬 Chat with Citizen AI")

        # User input section
        col1, col2 = st.columns([3, 1])

        with col1:
            user_name = st.text_input("👤 Your Name:", value="Citizen", key="user_name")

        with col2:
            if st.session_state.model_loaded:
                st.success("🟢 Ready")
            else:
                st.error("🔴 Load Model")

        # Chat input
        user_input = st.text_area(
            "💭 Your Message:",
            placeholder="Ask about government services, policies, or share your concerns...",
            height=100,
            key="chat_input"
        )

        col1, col2, col3 = st.columns([1, 1, 2])

        with col1:
            send_button = st.button("📤 Send", type="primary", use_container_width=True)

        with col2:
            clear_chat = st.button("🧹 Clear Chat", use_container_width=True)

        # Process input
        if send_button and user_input.strip():
            if st.session_state.model_loaded:
                with st.spinner("🤔 Thinking..."):
                    response = generate_response(user_input, user_name)
                st.rerun()
            else:
                st.error("Please initialize the model first!")

        if clear_chat:
            st.session_state.conversation_history = []
            st.rerun()

        # Display conversation
        st.markdown("### 💬 Conversation")

        if st.session_state.conversation_history:
            for entry in reversed(st.session_state.conversation_history[-10:]):  # Show last 10
                # User message
                st.markdown(f"""
                <div class="chat-message user-message">
                    <strong>👤 {entry['user_name']}:</strong><br>
                    {entry['user']}
                    <br><small>🕒 {entry['timestamp'].strftime('%H:%M:%S')} |
                    {entry['sentiment']['emoji']} {entry['sentiment']['label']} |
                    🏷️ {', '.join(entry['topics'])}</small>
                </div>
                """, unsafe_allow_html=True)

                # Assistant message
                st.markdown(f"""
                <div class="chat-message assistant-message">
                    <strong>🤖 Citizen AI:</strong><br>
                    {entry['assistant']}
                </div>
                """, unsafe_allow_html=True)
        else:
            st.info("👋 Start a conversation by sending a message!")

    with tab2:
        st.markdown("## 📊 Real-Time Dashboard")

        # Key metrics
        col1, col2, col3, col4 = st.columns(4)

        with col1:
            st.metric(
                "📱 Total Conversations",
                len(st.session_state.conversation_history)
            )

        with col2:
            if st.session_state.sentiment_data:
                avg_sentiment = np.mean([s['sentiment_score'] for s in st.session_state.sentiment_data])
                st.metric(
                    "😊 Average Sentiment",
                    f"{avg_sentiment:.2f}"
                )
            else:
                st.metric("😊 Average Sentiment", "0.00")

        with col3:
            active_topics = len(st.session_state.topics_discussed)
            st.metric(
                "🏷️ Active Topics",
                active_topics
            )

        with col4:
            recent_conversations = len([
                c for c in st.session_state.conversation_history
                if c['timestamp'] > datetime.now() - timedelta(hours=1)
            ])
            engagement_score = min(recent_conversations * 10, 100)
            st.metric(
                "🔥 Engagement Score",
                f"{engagement_score}/100"
            )

        # Sentiment distribution
        if st.session_state.sentiment_data:
            st.markdown("### 📊 Sentiment Distribution")

            sentiment_counts = Counter([s['sentiment_label'] for s in st.session_state.sentiment_data])

            col1, col2, col3 = st.columns(3)

            with col1:
                st.metric("😊 Positive", sentiment_counts.get('Positive', 0))
            with col2:
                st.metric("😐 Neutral", sentiment_counts.get('Neutral', 0))
            with col3:
                st.metric("😟 Negative", sentiment_counts.get('Negative', 0))

        # Top concerns
        if st.session_state.topics_discussed:
            st.markdown("### 🔝 Top Citizen Concerns")

            top_topics = Counter(st.session_state.topics_discussed).most_common(5)

            for i, (topic, count) in enumerate(top_topics, 1):
                st.markdown(f"**{i}. {topic.title()}**: {count} mentions")

    with tab3:
        st.markdown("## 📈 Advanced Analytics")

        col1, col2 = st.columns(2)

        with col1:
            # Sentiment chart
            sentiment_fig = create_sentiment_chart()
            st.plotly_chart(sentiment_fig, use_container_width=True)

        with col2:
            # Topics chart
            topics_fig = create_topics_chart()
            st.plotly_chart(topics_fig, use_container_width=True)

        # Detailed data table
        if st.session_state.conversation_history:
            st.markdown("### 📋 Conversation Details")

            # Create DataFrame
            data = []
            for entry in st.session_state.conversation_history:
                data.append({
                    'Time': entry['timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
                    'Citizen': entry['user_name'],
                    'Message Length': len(entry['user']),
                    'Sentiment': entry['sentiment']['label'],
                    'Score': round(entry['sentiment']['score'], 3),
                    'Topics': ', '.join(entry['topics'])
                })

            df = pd.DataFrame(data)
            st.dataframe(df, use_container_width=True)

            # Export option
            csv = df.to_csv(index=False)
            st.download_button(
                label="📥 Download Conversation Data",
                data=csv,
                file_name=f"citizen_ai_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv"
            )

if __name__ == "__main__":
    main()

# Run Streamlit
!streamlit run citizen_ai_streamlit.py #&
# import subprocess
# subprocess.run(["streamlit", "run", "citizen_ai_streamlit.py"])



# # Create public tunnel
# from pyngrok import ngrok
# public_url = ngrok.connect(8501)
# print(f"Public URL: {public_url}")
'''

# Write the app code to a file
with open('citizen_ai_app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

print("✅ Main application file 'citizen_ai_app.py' created successfully!")
print("📁 File size:", len(app_code), "characters")

✅ Main application file 'citizen_ai_app.py' created successfully!
📁 File size: 21695 characters


In [ ]:
# CELL 3: Setup ngrok for public access
#!pip install pyngrok
from pyngrok import ngrok
import os

# IMPORTANT: Replace 'YOUR_NGROK_TOKEN_HERE' with your actual token from ngrok.com
NGROK_TOKEN = "2yrFE9nfLipk5px1qItu6xTbLTR_3F18YPf9j15cDSNq7zR1a"  # 👈 PASTE YOUR TOKEN HERE

# Set authentication token
try:
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ ngrok authentication token set successfully!")
except Exception as e:
    print(f"❌ Error setting ngrok token: {e}")
    print("🔑 Please make sure you've replaced 'YOUR_NGROK_TOKEN_HERE' with your actual token")

✅ ngrok authentication token set successfully!


In [1]:
# FIXED DEPLOYMENT SCRIPT - Replace Cell 4 with this code

import subprocess
import threading
import time
from pyngrok import ngrok
import sys
import os

print("🔄 Stopping previous deployment...")
ngrok.kill()

print("🔧 Fixing Streamlit installation...")
# Ensure Streamlit is properly installed and accessible
try:
    # Reinstall streamlit with explicit path
    result = subprocess.run([sys.executable, "-m", "pip", "install", "streamlit", "--upgrade", "--force-reinstall"],
                          capture_output=True, text=True, timeout=120)
    print("✅ Streamlit installation completed")

    # Verify streamlit installation
    result = subprocess.run([sys.executable, "-m", "streamlit", "--version"],
                          capture_output=True, text=True, timeout=30)
    if result.returncode == 0:
        print(f"✅ Streamlit version: {result.stdout.strip()}")
    else:
        print(f"⚠️  Streamlit verification: {result.stderr}")

except Exception as e:
    print(f"❌ Error with Streamlit installation: {e}")

print("🚀 Starting Citizen AI Platform...")

# Function to run Streamlit using Python module approach
def run_streamlit():
    """Run Streamlit using python -m streamlit approach"""
    try:
        # Use python -m streamlit instead of direct streamlit command
        cmd = [
            sys.executable, "-m", "streamlit", "run", "citizen_ai_app.py",
            "--server.port", "8501",
            "--server.headless", "true",
            "--server.fileWatcherType", "none",
            "--browser.gatherUsageStats", "false",
            "--server.enableCORS", "false",
            "--server.enableXsrfProtection", "false"
        ]

        print(f"🔧 Running command: {' '.join(cmd)}")
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Monitor the process
        for line in iter(process.stdout.readline, ''):
            if line.strip():
                print(f"📡 Streamlit: {line.strip()}")
            if "Network URL" in line or "External URL" in line:
                break

    except Exception as e:
        print(f"❌ Error running Streamlit: {e}")

# Start Streamlit in background thread
print("📡 Starting Streamlit server...")
streamlit_thread = threading.Thread(target=run_streamlit, daemon=True)
streamlit_thread.start()

# Wait longer for Streamlit to fully start
print("⏳ Waiting for Streamlit to initialize...")
time.sleep(25)

# Test if Streamlit is running by checking the port
import socket
def check_port(host, port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(3)
        result = sock.connect_ex((host, port))
        sock.close()
        return result == 0
    except:
        return False

# Check if Streamlit is running
if check_port('localhost', 8501):
    print("✅ Streamlit server is responding on port 8501")
else:
    print("❌ Streamlit server not responding. Trying alternative approach...")

    # Alternative: Run streamlit directly with full path
    streamlit_path = subprocess.run([sys.executable, "-c", "import streamlit; print(streamlit.__file__)"],
                                  capture_output=True, text=True)
    if streamlit_path.returncode == 0:
        streamlit_dir = os.path.dirname(streamlit_path.stdout.strip())
        streamlit_script = os.path.join(streamlit_dir, "web", "cli.py")

        # Try running streamlit CLI directly
        def run_streamlit_alt():
            try:
                subprocess.run([sys.executable, streamlit_script, "run", "citizen_ai_app.py",
                              "--server.port", "8501", "--server.headless", "true"])
            except Exception as e:
                print(f"Alternative streamlit failed: {e}")

        alt_thread = threading.Thread(target=run_streamlit_alt, daemon=True)
        alt_thread.start()
        time.sleep(15)

# Create ngrok tunnel
try:
    print("🌐 Creating public tunnel...")

    # Create tunnel with retry logic
    max_retries = 3
    tunnel = None

    for attempt in range(max_retries):
        try:
            tunnel = ngrok.connect(8501, proto="http", bind_tls=True)
            break
        except Exception as e:
            print(f"❌ Tunnel attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(5)
                continue
            else:
                raise e

    if tunnel:
        print("\n" + "="*70)
        print("🎉 CITIZEN AI PLATFORM IS DEPLOYED!")
        print("="*70)
        print(f"🌐 Public URL: {tunnel.public_url}")
        print(f"🔗 Click here: {tunnel.public_url}")
        print("="*70)
        print("\n🎯 Application Features:")
        print("✅ Real-Time AI Assistant (IBM Granite-3.3-2B)")
        print("✅ Citizen Sentiment Analysis")
        print("✅ Dynamic Dashboard & Analytics")
        print("✅ Personalized Contextual Responses")
        print("✅ T4 GPU Acceleration")
        print("\n💡 First-Time Setup:")
        print("• The AI model will load when you first visit (2-3 minutes)")
        print("• Set your location and interests in the sidebar")
        print("• Try asking: 'How do I register to vote?'")
        print("\n⚠️  IMPORTANT: Keep this cell running to maintain access!")
        print("="*70)

        # Keep tunnel alive with better error handling
        try:
            while True:
                time.sleep(30)
                # Check if tunnel is still active
                try:
                    tunnels = ngrok.get_tunnels()
                    if not tunnels:
                        print("🔄 Tunnel disconnected, recreating...")
                        tunnel = ngrok.connect(8501, proto="http", bind_tls=True)
                        print(f"🌐 New URL: {tunnel.public_url}")
                    else:
                        print(f"🔄 App running... Access: {tunnel.public_url}")
                except Exception as tunnel_check_error:
                    print(f"⚠️  Tunnel check error: {tunnel_check_error}")

        except KeyboardInterrupt:
            print("\n🛑 Shutting down Citizen AI Platform...")
            ngrok.kill()

except Exception as e:
    print(f"❌ Error creating tunnel: {e}")
    print("\n🔧 Troubleshooting Steps:")
    print("1. Make sure your ngrok token is set correctly")
    print("2. Try restarting the runtime: Runtime → Restart and run all")
    print("3. Check if all packages installed correctly")

    # Show alternative local access
    print(f"\n💡 Alternative: If you're running locally, try: http://localhost:8501")

ModuleNotFoundError: No module named 'pyngrok'

In [ ]:
# VERIFICATION SCRIPT - Run this first to diagnose issues

import sys
import subprocess
import os
import importlib.util

print("🔍 CITIZEN AI PLATFORM - DIAGNOSTIC CHECK")
print("="*50)

# Check Python version
print(f"🐍 Python Version: {sys.version}")

# Check if we're in Colab
try:
    import google.colab
    print("✅ Running in Google Colab")
    IN_COLAB = True
except ImportError:
    print("❌ Not running in Google Colab")
    IN_COLAB = False

# Check GPU availability
try:
    import torch
    print(f"🚀 PyTorch Version: {torch.__version__}")
    print(f"🎯 CUDA Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🔥 GPU Device: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
except Exception as e:
    print(f"❌ PyTorch/CUDA Error: {e}")

print("\n🔧 PACKAGE INSTALLATION CHECK")
print("="*50)

# List of required packages
required_packages = [
    'streamlit', 'transformers', 'torch', 'plotly',
    'textblob', 'wordcloud', 'matplotlib', 'seaborn',
    'pandas', 'numpy', 'pyngrok'
]

missing_packages = []
for package in required_packages:
    try:
        spec = importlib.util.find_spec(package)
        if spec is not None:
            print(f"✅ {package}")
        else:
            print(f"❌ {package} - NOT FOUND")
            missing_packages.append(package)
    except Exception as e:
        print(f"❌ {package} - ERROR: {e}")
        missing_packages.append(package)

# Install missing packages
if missing_packages:
    print(f"\n🔄 Installing missing packages: {missing_packages}")
    for package in missing_packages:
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", package],
                         check=True, capture_output=True)
            print(f"✅ Installed {package}")
        except Exception as e:
            print(f"❌ Failed to install {package}: {e}")

print("\n📁 FILE CHECK")
print("="*50)

# Check if main app file exists
if os.path.exists('citizen_ai_app.py'):
    print("✅ citizen_ai_app.py exists")
    file_size = os.path.getsize('citizen_ai_app.py')
    print(f"📏 File size: {file_size} bytes")

    # Check if file is not empty and has expected content
    with open('citizen_ai_app.py', 'r') as f:
        content = f.read()
        if 'streamlit' in content and 'granite' in content:
            print("✅ File contains expected content")
        else:
            print("❌ File may be corrupted or incomplete")
else:
    print("❌ citizen_ai_app.py NOT FOUND")
    print("🔄 Creating the file now...")

    # Recreate the file if missing
    app_code = '''
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import plotly.express as px
import pandas as pd
import numpy as np
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from datetime import datetime
import re
from collections import defaultdict, Counter

st.set_page_config(
    page_title="Citizen AI - Test",
    page_icon="🏛️",
    layout="wide"
)

st.title("🏛️ Citizen AI - Platform Test")
st.write("If you can see this, the basic setup is working!")

# Test basic functionality
st.header("🔧 System Check")

# Check GPU
if torch.cuda.is_available():
    st.success(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
else:
    st.warning("⚠️ No GPU detected")

# Simple test
st.header("💬 Simple Test")
user_input = st.text_input("Enter a test message:")
if user_input:
    st.write(f"You said: {user_input}")

    # Test sentiment analysis
    blob = TextBlob(user_input)
    sentiment = "Positive" if blob.sentiment.polarity > 0 else "Negative" if blob.sentiment.polarity < 0 else "Neutral"
    st.write(f"Sentiment: {sentiment}")

st.success("🎉 Basic functionality test passed!")
'''

    with open('citizen_ai_app.py', 'w') as f:
        f.write(app_code)
    print("✅ Created basic test version of citizen_ai_app.py")

print("\n🌐 STREAMLIT CHECK")
print("="*50)

# Test Streamlit command
try:
    result = subprocess.run([sys.executable, "-m", "streamlit", "--version"],
                          capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print(f"✅ Streamlit command works: {result.stdout.strip()}")
    else:
        print(f"❌ Streamlit command failed: {result.stderr}")
except Exception as e:
    print(f"❌ Streamlit command error: {e}")

print("\n🔑 NGROK CHECK")
print("="*50)

try:
    from pyngrok import ngrok
    tunnels = ngrok.get_tunnels()
    print(f"✅ ngrok imported successfully")
    print(f"📡 Active tunnels: {len(tunnels)}")
    for tunnel in tunnels:
        print(f"🌐 Tunnel: {tunnel}")
except Exception as e:
    print(f"❌ ngrok error: {e}")

print("\n📋 RECOMMENDATIONS")
print("="*50)

if IN_COLAB:
    print("✅ You're in Google Colab - Good!")
    print("💡 Make sure you have T4 GPU enabled")
    print("💡 Get your ngrok token from https://ngrok.com")
else:
    print("⚠️  You're not in Google Colab")
    print("💡 This setup is optimized for Google Colab")

print("\n🚀 NEXT STEPS")
print("="*50)
print("1. If all checks pass, run the fixed deployment script")
print("2. Make sure your ngrok token is set correctly")
print("3. Wait for the 'PLATFORM IS DEPLOYED' message")
print("4. Click the provided public URL")

print("\n" + "="*50)
print("🔍 DIAGNOSTIC COMPLETE")
print("="*50)

🔍 CITIZEN AI PLATFORM - DIAGNOSTIC CHECK
🐍 Python Version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
✅ Running in Google Colab
🚀 PyTorch Version: 2.6.0+cu124
🎯 CUDA Available: True
🔥 GPU Device: Tesla T4
💾 GPU Memory: 15.8 GB

🔧 PACKAGE INSTALLATION CHECK
❌ streamlit - NOT FOUND
✅ transformers
✅ torch
✅ plotly
✅ textblob
✅ wordcloud
✅ matplotlib
✅ seaborn
✅ pandas
✅ numpy
✅ pyngrok

🔄 Installing missing packages: ['streamlit']
✅ Installed streamlit

📁 FILE CHECK
✅ citizen_ai_app.py exists
📏 File size: 16354 bytes
✅ File contains expected content

🌐 STREAMLIT CHECK
✅ Streamlit command works: Streamlit, version 1.46.0

🔑 NGROK CHECK
✅ ngrok imported successfully
📡 Active tunnels: 0

📋 RECOMMENDATIONS
✅ You're in Google Colab - Good!
💡 Make sure you have T4 GPU enabled
💡 Get your ngrok token from https://ngrok.com

🚀 NEXT STEPS
1. If all checks pass, run the fixed deployment script
2. Make sure your ngrok token is set correctly
3. Wait for the 'PLATFORM IS DEPLOYED' message
4. Cl